In [ ]:
from kafka import KafkaProducer, KafkaConsumer
import requests
import json
import time
from datetime import datetime
import logging

# Cấu hình logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Cấu hình thông số
CONFIG = {
    "API_KEY": "0456962f3b8714167ed9ab89c38dc02e",
    "lat": "10.7758439",
    "lon": "106.7017555",
    "kafka_bootstrap_servers": "kafka:9092",  # Địa chỉ Kafka broker
    "kafka_topic": "weather_data",  # Tên Topic
    "normal_interval": 5,  # Thu thập mỗi 5 giây
    "error_retry_interval": 5  # Chờ 5 giây khi có lỗi
}

   
def create_kafka_producer():
    """Tạo và trả về Kafka producer"""
    # Tạo Producer
    try:
        producer = KafkaProducer(
            bootstrap_servers=CONFIG['kafka_bootstrap_servers'],
            value_serializer=lambda v: json.dumps(v).encode('utf-8'),
            acks='all',
            retries=3
        )
        logger.info("Kafka producer được tạo thành công")
        return producer
    except Exception as e:
        logger.error(f"Lỗi khi tạo Kafka producer: {e}")
        return None

def fetch_weather_data():
    """Thu thập dữ liệu thời tiết từ API"""
    url = f"https://api.openweathermap.org/data/2.5/weather?lat={CONFIG['lat']}&lon={CONFIG['lon']}&appid={CONFIG['API_KEY']}"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        
        # Thêm timestamp vào dữ liệu
        data['timestamp'] = datetime.now().isoformat()
        return data
    
    except requests.exceptions.RequestException as e:
        logger.error(f"Lỗi khi kết nối API thời tiết: {e}")
        return None
    except Exception as e:
        logger.error(f"Lỗi: {e}")
        return None

def send_to_kafka(producer, data):
    """Gửi dữ liệu đến Kafka"""
    try:
        prod = producer.send(
            CONFIG['kafka_topic'],
            value=data
        )
        # Chờ xác nhận từ Kafka
        prod.get(timeout=10)
        logger.info(f"Đã gửi dữ liệu đến Kafka topic {CONFIG['kafka_topic']} || timestamp : {data['timestamp']}")
        return True
    except Exception as e:
        logger.error(f"Lỗi khi gửi dữ liệu đến Kafka: {e}")
        return False

def main():
    """Chương trình chính"""
    # Phải tạo Producer cho bằng được rồi mới tới các bước tiếp
    producer = create_kafka_producer()
    while not producer:
        time.sleep(CONFIG['error_retry_interval'])
        producer = create_kafka_producer()
        return
    
    while True:
        try:
            # Thu thập dữ liệu thời tiết
            weather_data = fetch_weather_data()
            
            if weather_data:
                # Gửi dữ liệu đến Kafka
                success = send_to_kafka(producer, weather_data)
                
                if success:
                    # Chờ interval bình thường nếu thành công
                    time.sleep(CONFIG['normal_interval'])
                else:
                    # Chờ interval ngắn hơn nếu có lỗi
                    time.sleep(CONFIG['error_retry_interval'])
            else:
                # Chờ interval ngắn hơn nếu không lấy được dữ liệu
                time.sleep(CONFIG['error_retry_interval'])
                
        except Exception as e:
            logger.error(f"Lỗi không mong muốn: {e}")
            time.sleep(CONFIG['error_retry_interval'])

if __name__ == "__main__":
    logger.info("Bắt đầu chương trình thu thập dữ liệu thời tiết")
    main()


2025-04-28 10:05:22,870 - INFO - Bắt đầu chương trình thu thập dữ liệu thời tiết
2025-04-28 10:05:22,875 - INFO - <BrokerConnection node_id=bootstrap-0 host=kafka:9092 <connecting> [IPv4 ('172.18.0.3', 9092)]>: connecting to kafka:9092 [('172.18.0.3', 9092) IPv4]
2025-04-28 10:05:22,876 - INFO - Probing node bootstrap-0 broker version
2025-04-28 10:05:22,905 - INFO - <BrokerConnection node_id=bootstrap-0 host=kafka:9092 <connecting> [IPv4 ('172.18.0.3', 9092)]>: Connection complete.
2025-04-28 10:05:23,009 - INFO - Broker version identified as 1.0.0
2025-04-28 10:05:23,011 - INFO - Set configuration api_version=(1, 0, 0) to skip auto check_version requests on startup
2025-04-28 10:05:23,015 - INFO - Kafka producer được tạo thành công
2025-04-28 10:05:23,204 - INFO - <BrokerConnection node_id=0 host=037c90c57af2:9092 <connecting> [IPv4 ('172.18.0.3', 9092)]>: connecting to 037c90c57af2:9092 [('172.18.0.3', 9092) IPv4]
2025-04-28 10:05:23,207 - INFO - <BrokerConnection node_id=0 host=037

KeyboardInterrupt: 

2025-04-28 10:09:21,207 - INFO - Closing the Kafka producer with 0 secs timeout.
2025-04-28 10:09:21,207 - INFO - Proceeding to force close the producer since pending requests could not be completed within timeout 0.
2025-04-28 10:09:21,255 - INFO - <BrokerConnection node_id=0 host=037c90c57af2:9092 <connected> [IPv4 ('172.18.0.3', 9092)]>: Closing connection. 


In [3]:
import requests
API_KEY = "0456962f3b8714167ed9ab89c38dc02e"
LIMIT = 10
CITY_NAME = "Da Nang"
URL = f"http://api.openweathermap.org/geo/1.0/direct?q={CITY_NAME}&limit={LIMIT}&appid={API_KEY}"

response = requests.get(URL)
response.raise_for_status()
data = response.json()
print(data)

[{'name': 'Da Nang', 'local_names': {'ko': '다낭', 'km': 'ដាណាំង', 'en': 'Da Nang', 'pt': 'Da Nang', 'zh': '峴港', 'ru': 'Дананг', 'vi': 'Thành phố Đà Nẵng', 'eo': 'Danango', 'ja': 'ダナン'}, 'lat': 16.068, 'lon': 108.212, 'country': 'VN'}, {'name': 'Đà Nẵng', 'local_names': {'ko': '다낭', 'vi': 'Đà Nẵng'}, 'lat': 20.9147215, 'lon': 106.6735556, 'country': 'VN'}, {'name': 'Tai Kek', 'local_names': {'zh': '大乪', 'en': 'Tai Kek'}, 'lat': 22.4167221, 'lon': 114.0691847, 'country': 'CN', 'state': 'Hong Kong'}]


In [4]:
for d in data:
    print(d)

{'name': 'Da Nang', 'local_names': {'ko': '다낭', 'km': 'ដាណាំង', 'en': 'Da Nang', 'pt': 'Da Nang', 'zh': '峴港', 'ru': 'Дананг', 'vi': 'Thành phố Đà Nẵng', 'eo': 'Danango', 'ja': 'ダナン'}, 'lat': 16.068, 'lon': 108.212, 'country': 'VN'}
{'name': 'Đà Nẵng', 'local_names': {'ko': '다낭', 'vi': 'Đà Nẵng'}, 'lat': 20.9147215, 'lon': 106.6735556, 'country': 'VN'}
{'name': 'Tai Kek', 'local_names': {'zh': '大乪', 'en': 'Tai Kek'}, 'lat': 22.4167221, 'lon': 114.0691847, 'country': 'CN', 'state': 'Hong Kong'}
